# Logistic Regression

1. Design model (input,output size, forward pass)
2. construct loss and optimizer
3. training loop
    - forward pass: compute prediction loss
    - backward pass: gradients
    - update weights

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
bc = datasets.load_breast_cancer()
x,y = bc.data, bc.target

In [4]:
# prepare data
n_samples, n_features = x.shape
x.shape

(569, 30)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1234)

In [6]:
# scale
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [7]:
x_train, y_train

(array([[-0.36180827, -0.26521011, -0.31715702, ..., -0.07967528,
         -0.52798733,  0.2506337 ],
        [-0.8632675 ,  0.71560604, -0.85646012, ..., -0.76980239,
          0.44312729, -0.20987332],
        [-0.4334453 ,  0.32513895, -0.41286667, ..., -0.06601541,
         -1.1169427 ,  0.0329492 ],
        ...,
        [-0.479293  , -0.17689018, -0.45697634, ..., -0.20261414,
          0.18670009,  0.17414996],
        [ 1.16835876, -0.15364809,  1.17466524, ...,  0.26789258,
          0.19828067, -0.23394164],
        [-0.40765597, -1.29715887, -0.42826344, ..., -0.78042674,
         -0.88036793, -0.80355834]]),
 array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
   

In [8]:
# numpy to torch
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [9]:
x_train, y_train

(tensor([[-0.3618, -0.2652, -0.3172,  ..., -0.0797, -0.5280,  0.2506],
         [-0.8633,  0.7156, -0.8565,  ..., -0.7698,  0.4431, -0.2099],
         [-0.4334,  0.3251, -0.4129,  ..., -0.0660, -1.1169,  0.0329],
         ...,
         [-0.4793, -0.1769, -0.4570,  ..., -0.2026,  0.1867,  0.1741],
         [ 1.1684, -0.1536,  1.1747,  ...,  0.2679,  0.1983, -0.2339],
         [-0.4077, -1.2972, -0.4283,  ..., -0.7804, -0.8804, -0.8036]]),
 tensor([1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
         0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
         1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
         1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.,
         1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
         1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
         0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,

In [10]:
# reshape
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [19]:
# 1) Logistic Regression model --> f = wx+b with sigmoid at end
# LogisticRegression class inheritated from nn.Module
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        #For fully connected layer, we use nn.Linear
        self.linear = nn.Linear(n_input_features, 1) #layer 1
        
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

# class object
model = LogisticRegression(n_features)

In [20]:
# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss() #<-- Binary cross entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [22]:
# 3) training loop
num_epochs = 100

for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(x_train)
    loss = criterion(y_predicted, y_train)
    
    # backward pass
    loss.backward()
    
    # updates
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch+1)%10 == 0:
        print(f'epoch: {epoch+1}, loss={loss.item():.4f}')

epoch: 10, loss=0.2282
epoch: 20, loss=0.2194
epoch: 30, loss=0.2115
epoch: 40, loss=0.2045
epoch: 50, loss=0.1981
epoch: 60, loss=0.1924
epoch: 70, loss=0.1871
epoch: 80, loss=0.1823
epoch: 90, loss=0.1779
epoch: 100, loss=0.1737


In [23]:
# evaluation
with torch.no_grad(): #<--avoid tracking gradient calc
    y_predicted = model(x_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(acc)

tensor(0.9123)
